# import #

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# fnc #

In [ ]:
#debug
def train_test_debug():
    if train_set["text"][0] == training_data1["text"][0]:
        print("train set is ***1***")
    
    if train_set["text"][0] == training_data2["text"][0]:
        print("train set is ***2***")
        
    if test_set["text"][0] == testing_data1["text"][0]:
        print("test set is ***1***")
    
    if test_set["text"][0] == testing_data2["text"][0]:
        print("test set is ***2***")

In [ ]:
def clean_text(text):
    text = str(text)
    if "," in text:
        return str(text.replace(",",""))
    if "." in text:
        return str(text.replace(".",""))
    if "।" in text:
        return str(text.replace("।",""))
    if "|" in text:
        return str(text.replace("|",""))
    return str(text)

In [ ]:
def get_emotion(emotion):
    if emotion == "happy":
        return 0
    elif emotion == "sad":
        return 1
    elif emotion == "angry":
        return 2
    elif emotion == "disgust":
        return 3
    elif emotion == "fear":
        return 4

# data #

**dataset1**

In [ ]:
training_data1 = pd.read_excel("all_data/training_DS1.xlsx")
print(training_data1.groupby("reaction").apply(list))
print(f"shape = {training_data1.shape}")
print(f"dtype = {training_data1.dtypes}")
print(f"value count = \n{training_data1['reaction'].value_counts()}")

training_list1 = training_data1.values.tolist()
print("\n")
testing_data1 = pd.read_excel("all_data/testing_DS1.xlsx")
print(testing_data1.groupby("reaction").apply(list))
print(f"shape = {testing_data1.shape}")
print(f"dtype = {testing_data1.dtypes}")
print(f"value count = \n{testing_data1['reaction'].value_counts()}")

testing_list1 = training_data1.values.tolist()

In [ ]:
training_data1["text"] = training_data1["text"].apply(clean_text)
training_data1["label"] = training_data1["reaction"].apply(get_emotion)
training_data1["num_of_words"] = training_data1["text"].apply(lambda x:len(str(x).split()))

print("\n split \n")

testing_data1["text"] = testing_data1["text"].apply(clean_text)
testing_data1["label"] = testing_data1["reaction"].apply(get_emotion)
testing_data1["num_of_words"] = testing_data1["text"].apply(lambda x:len(str(x).split()))

**dataset2**

In [ ]:
training_data2 = pd.read_excel("all_data/training_DS2.xlsx")
print(training_data2.groupby("reaction").apply(list))
print(f"shape = {training_data2.shape}")
print(f"dtype = {training_data2.dtypes}")
print(f"value count = \n{training_data2['reaction'].value_counts()}")

training_list2 = training_data2.values.tolist()
print("\n")
testing_data2 = pd.read_excel("all_data/testing_DS2.xlsx")
print(testing_data2.groupby("reaction").apply(list))
print(f"shape = {testing_data2.shape}")
print(f"dtype = {testing_data2.dtypes}")
print(f"value count = \n{testing_data2['reaction'].value_counts()}")

testing_list2 = training_data2.values.tolist()

In [ ]:
training_data2["text"] = training_data2["text"].apply(clean_text)
training_data2["label"] = training_data2["reaction"].apply(get_emotion)
training_data2["num_of_words"] = training_data2["text"].apply(lambda x:len(str(x).split()))
print(training_data2.groupby("reaction").apply(list))

print("\n split \n")

testing_data2["text"] = testing_data2["text"].apply(clean_text)
testing_data2["label"] = testing_data2["reaction"].apply(get_emotion)
testing_data2["num_of_words"] = testing_data2["text"].apply(lambda x:len(str(x).split()))
print(training_data2.groupby("reaction").apply(list))

**select dataset**

In [ ]:
train_set = None
test_set = None

In [ ]:
def select_train_test(train,test):
    global train_set
    global test_set
    if train == 1 and test == 1:
        print("train = 1 test = 1 loaded")
        print(testing_data1.groupby("reaction").apply(list))
        train_set = training_data1.drop(columns=["reaction","num_of_words"])
        test_set = testing_data1.drop(columns=["reaction","num_of_words"])
    elif train == 2 and test == 2:
        print("train = 2 test = 2 loaded")
        print(training_data2.groupby("reaction").apply(list))
        train_set = training_data2.drop(columns=["reaction","num_of_words"])
        test_set = testing_data2.drop(columns=["reaction","num_of_words"])
    elif train == 1 and test == 2:
        print("train = 1 test = 2 loaded")
        train_set = training_data1.drop(columns=["reaction","num_of_words"])
        test_set = testing_data2.drop(columns=["reaction","num_of_words"])
    elif train == 2 and test == 1:
        print("train = 2 test = 1 loaded")
        train_set = training_data2.drop(columns=["reaction","num_of_words"])
        test_set = testing_data1.drop(columns=["reaction","num_of_words"])
    else:
        print("choose between train 1/2 and test 1/2")
        
    print(f"value count = \n{train_set['label'].value_counts()}")
    print(f"value count = \n{test_set['label'].value_counts()}")


In [2]:
from simpletransformers.classification import ClassificationModel

# Select train test #

In [ ]:
select_train_test(1,1)
train_test_debug()

In [ ]:
select_train_test(2,2)
train_test_debug()

In [ ]:
select_train_test(1,2)
train_test_debug()

In [ ]:
select_train_test(2,1)
train_test_debug()

In [3]:
train_set = pd.read_csv("csv_data/train.csv")
test_set = pd.read_csv("csv_data/test_set_2.csv")
# validate_set = pd.read_csv("csv_data/test_set_2.csv")

# Model "monsoon-nlp/bangla-electra" #

In [ ]:
model = ClassificationModel('bert', 'monsoon-nlp/bangla-electra', num_labels=5, use_cuda=True, args={
    'reprocess_input_data': True,
    'use_cached_eval_features': False,
    'overwrite_output_dir': True,
    'num_train_epochs': 3,
    'silent': True
}) # , weight=[2.5, 1.0]
model.train_model(train_set.sample(frac=1))

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_set)
bads = {}
for pred in wrong_predictions:
    if pred.label in bads:
        bads[pred.label] += 1
    else:
        bads[pred.label] = 1
print("wrong predictions:")
print(str(len(wrong_predictions)) + ' wrong out of ' + str(len(test_set)))
print(f"accuracy = {(len(test_set)-len(wrong_predictions))/len(test_set)*100}")
bads

In [ ]:
train_test_debug()

# Report monsoon-nlp/bangla-electra #

train set is ***1***
test set is ***1***
wrong predictions:
474 wrong out of 1032
accuracy = 54.06976744186046

train set is ***2***
test set is ***2***
wrong predictions:
628 wrong out of 1265
accuracy = 50.35573122529644

train set is ***1***
test set is ***2***
wrong predictions:
874 wrong out of 1265
accuracy = 30.909090909090907

train set is ***2***
test set is ***1***
wrong predictions:
699 wrong out of 1032
accuracy = 32.26744186046512

# Model "sagorsarker/bangla-bert-base" #

In [ ]:
train_test_debug()

In [4]:
bert2 = ClassificationModel('bert', 'sagorsarker/bangla-bert-base', num_labels=5, use_cuda=True, args={
    'reprocess_input_data': True,
    'use_cached_eval_features': False,
    'overwrite_output_dir': True,
    'num_train_epochs': 3,
    'silent': True
})
bert2.train_model(train_set.sample(frac=1))

Some weights of the model checkpoint at sagorsarker/bangla-bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

KeyboardInterrupt: 

In [ ]:
result, model_outputs, wrong_predictions = bert2.eval_model(test_set)
bads = {}
for pred in wrong_predictions:
    if pred.label in bads:
        bads[pred.label] += 1
    else:
        bads[pred.label] = 1
print("wrong predictions:")
print(str(len(wrong_predictions)) + ' wrong out of ' + str(len(test_set)))
print(f"accuracy = {(len(test_set)-len(wrong_predictions))/len(test_set)*100}")
bads

In [ ]:
train_test_debug()

# Report sagorsarker/bangla-bert-base #

train set is ***1***
test set is ***1***
wrong predictions:
214 wrong out of 1032
accuracy = 79.26356589147287

train set is ***2***
test set is ***2***
wrong predictions:
185 wrong out of 1265
accuracy = 85.37549407114624

train set is ***1***
test set is ***2***
wrong predictions:
722 wrong out of 1265
accuracy = 42.92490118577075

train set is ***2***
test set is ***1***
wrong predictions:
578 wrong out of 1032
accuracy = 43.992248062015506

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
y_true = np.array(all_labels)
y_pred = np.array(all_preds)
print(precision_recall_fscore_support(y_true, y_pred, average='macro'))
print(precision_recall_fscore_support(y_true, y_pred, average='micro'))
print(precision_recall_fscore_support(y_true, y_pred, average='weighted'))

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print(classification_report(y_true, y_pred))